In [1]:
import pandas as pd

df = pd.read_csv('../Data/2019/2019_pre2.csv', encoding='utf-8-sig')
df.tail()

,Unnamed: 0,date,press,category,headline,url,text,img,N_list
128641,128641,2019/06/30,세계일보,"정치,국제",[WT논평]The reason why we must denuclearize Nort...,http://www.segye.com/content/html/2019/06/30/2...,By Robert R. Monroe (former director of the De...,[],"[['북한', '핵', '이유'], ['합의', '시바', '삐', '매듭', '필..."
128642,128642,2019/06/30,세계일보,"국제,정치","[사설] 日 경제보복 카드 우려, 정부는 전략적 대응 모색하길",http://www.segye.com/content/html/2019/06/30/2...,일본이 강제징용 판결에 대해 경제보복 카드를 꺼내든다고 한다. 일본 정부가 TV·스...,['//img.segye.com/content/image/2019/06/30/201...,"[['일본', '강제', '징용', '판결', '경제', '보복', '카드'], [..."
128643,128643,2019/06/30,세계일보,정치,[사설] 84일 만의 국회 본회의… ‘완전한 정상화’ 서두를 때다,http://www.segye.com/content/html/2019/06/30/2...,국회가 지난 28일 자유한국당을 포함한 여야 5당이 모두 참석한 가운데 본회의를 열...,['//img.segye.com/content/image/2019/06/28/201...,"[['국회', '자유한국당', '포함', '여야', '당', '참석', '가운데',..."
128644,128644,2019/06/30,세계일보,정치,"[사설] 남·북·미 정상 판문점 회동, 실질적 비핵화로 이어져야",http://www.segye.com/content/html/2019/06/30/2...,분단·냉전 상징서 역사적 만남 / 트럼프 “북·미 실무협상 할 것” / 北 비핵화 ...,['//img.segye.com/content/image/2019/06/30/201...,"[['분단', '냉전', '상징', '역사', '만남', '트럼프', '북', '미..."
128645,128645,2019/06/30,중앙일보,정치,"조국 ""사실상 종전선언""…靑참모들 일제히 ""역사적인 날""",https://news.joins.com/article/olink/23103851,"조국 청와대 민정수석이 30일 남북미 세 정상의 판문점 회동에 대해 ""세 지도자의 ...",['https://pds.joins.com/news/component/htmlpho...,"[['조국', '청와대', '민정', '수석', '남북미', '정상', '판문점',..."


In [46]:
import re
from collections import Counter
import math

def npmi(a, b, n, z):
    x = math.log((n*z)/(a*b))
    y = math.log(n/z)
    return x / y

docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []
    merge_token2 = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        context += arr

    #2 PMI 적용
    com += list(zip(context, context[1:]))
    n = len(context)
    cnt = Counter(context)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        if -1 < pmi < 1:
            nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+" "+key[1] for key, item in result if item >= 0.90]

    # 토큰을 합치는 내용
    for i in range(len(context)-1):
        for token in merge_token:
            cmp_str = ' '.join(context[i:i+2])
            if cmp_str == token:
                del context[i:i+2]
                context.insert(i, token)
    docs.append(context)

    if len(docs) % 1000 == 0:
        print(len(docs))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000


In [47]:
import tomotopy as tp

def extract_topic(docs):
    model = tp.LDAModel(k=7, alpha=1.0, eta=0.1, min_cf=3, min_df=1, tw=tp.TermWeight.PMI)
    for doc in docs:
        model.add_doc(doc) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    for j in range(0,100):
        model.train(10) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    result = []
    for k in range(model.k): #k는 토픽의 개수
        s = []
        res = model.get_topic_words(k, top_n=20) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        for w, p in res:
            s.append(w)
        result.append(', '.join(s))
    return result

In [48]:
result = extract_topic(docs)
i = 1
for topic in result:
    print(i, topic)
    i += 1

1 주택, 먼지, 미세, 지역, 분양, 아파트, 서울, 차, 차량, 가구, 가격, 대, 사업, 환경, 도시, 에너지, 버스, 전기, 안전, 개발
2 의원, 국회, 대통령, 대표, 당, 한국당, 원내대표, 청와대, 국민, 선거, 정치, 후보자, 민주당, 후보, 자유한국당, 문, 장관, 인사, 위원, 개혁
3 교육, 학생, 대학, 학교, 임금, 지원, 환자, 소득, 청년, 노조, 노동, 고용, 일자리, 취업, 교수, 평가, 유치원, 세, 교사, 장애인
4 수사, 검찰, 경찰, 혐의, 사건, 의혹, 재판, 구속, 조사, 김, 판사, 법원, 변호사, 승리, 영장, 검사, 버닝썬, 기소, 윤, 심
5 기업, 투자, 시장, 금융, 경제, 제품, 성장, 수출, 달러, 조, 분기, 금리, 대출, 은행, 회장, 브랜드, 고객, 상품, 판매, 산업
6 경찰, 여성, 마약, 사고, 혐의, 범행, 피해자, 남성, 사건, 폭행, 김, 발견, 조사, 성, 신고, 동물, 피해, 영상, 헝가리, 박
7 북한, 미국, 중국, 트럼프, 대통령, 회담, 핵, 일본, 정상, 미, 협상, 한국, 북, 제재, 김 위원장, 미사일, 위원장, 러시아, 북미, 남북
